# Chapter 3: RAG Part II: Chatting with Your Data

## Query Routing

Although using a single vector store is useful, the required data may live in a variety of data sources, including relational databases or other vector stores.

For example, you may have two vector stores: one for LangChain Python documentation and another for LangChain JS documentation. Given a user’s question, we would like to route the query to the appropriate inferred data source to retrieve relevant docs. _Query routing_ is a strategy used to forward a user’s query to the relevant data source.

### Semantic Routing

Unlike logical routing, _semantic routing_ involves embedding various prompts that represent various data sources alongside the user’s query and then performing vector similarity search to retrieve the most similar prompt.

The following is an example of semantic routing:

In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_deepseek import ChatDeepSeek
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

# define embedding model
hf_embedding = HuggingFaceEmbeddings(
    model="sentence-transformers/all-mpnet-base-v2", # use this model to perform the embedding
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)

llm = ChatDeepSeek(model="deepseek-chat", temperature=0)

# define prompts
physics_template = """You are a very smart physics professor. You are great at answering questions about physics in a concise and easy-to-understand manner. When you don't know the answer to a question, you admit that you don't know.
Here is a question:
{question}"""

math_template = """You're a very good mathematician. You're great at answering math questions. You're so good because you're able to break down hard problems into their component parts, answer the component parts, and then put them together to answer a broader question.
Here is a question:
{question}
"""

# embedd the prompts
prompt_templates = [physics_template, math_template]
prompt_embeddings = hf_embedding.embed_documents(texts=prompt_templates)

Define semantic router:

In [4]:
@chain
def prompt_router(query: str) -> PromptTemplate:
    # embed question
    query_embedding = hf_embedding.embed_query(text=query)

    # compute similarity
    similarity = cosine_similarity([query_embedding, ], prompt_embeddings)[0]
    # pick prompt most similar to the input question
    most_similar = prompt_templates[similarity.argmax()]

    return PromptTemplate.from_template(most_similar)

semantic_router = prompt_router | llm | StrOutputParser()

Run the model:

In [5]:
response = semantic_router.invoke(input="What's is a black hole?")
print(response)

Excellent question. Let's break it down.

A black hole is a region of space where gravity is so intense that nothing, not even light, can escape from it.

Think of it this way: every object has an "escape velocity," the speed you need to go to break free of its gravity. For Earth, that's about 11 km/s (25,000 mph). For a black hole, the escape velocity at its boundary is greater than the speed of light, which is the universe's ultimate speed limit. Since nothing can travel faster than light, nothing can escape.

Here are the key parts:

1.  **The Event Horizon:** This is the "point of no return," the boundary of the black hole. Once anything crosses this invisible spherical surface, it is inexorably pulled inward. We cannot see what happens inside.

2.  **The Singularity:** At the very center of the black hole, all the mass is thought to be crushed into a single point of infinite density. Our current laws of physics break down here, and we need a theory of quantum gravity (which we don